环境安装

In [1]:
!pip install moviepy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 388 kB 318 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 26.9 MB 2.5 MB/s eta 0:00:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110743 sha256=727dace4b3cb9e91a48d5e1ae360673f6a7841ed73cfd318571ccd43a5f4df82
  Stored in directory: /home/yolov5test/.cache/pip/wheels/fb/56/20/ea8039f0f19ff16db00131559d9290ff7cddf4711b64a3ff27
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6156 sha256=cc5c0e13c99115d736981b8fb8511ee617b09151b9bab64f2a93c32c9db9a31c
  Stored in directory: /home/yolov5test/.cache/pip/wheels/3a/91/2c/1c905394471c2f283b3f2997682e1d3d51fb35a089b08e923b
Successfully built moviepy proglog
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pi

随机加载、随机连接、随机选取背景音乐、结果输出

- 随机加载：将预处理输出后的视频（samples中的n个视频）进行load
- 随机连接：随机拼接（后期如果可以进行语义分析的话，这里就可以使用输入的文本进行匹配链接；改进Editing切片，再按照顺序拼接就行）
- 随机选取背景音乐：后续加入情感分析背景音乐，这里可以手动输入bgm（不建议手动输入，因为AI剪切出来的视频感情倾向模糊）

In [1]:
from moviepy.editor import *
import os, glob
import random
import argparse

def extract_clip(samples, output_duration):
	extract_duration = output_duration/len(samples)
	extracted_samples = []
	for s in samples:
		start_point = random.uniform(0, s.duration - extract_duration)
		end_point = start_point + extract_duration
		extracted_sample = s.subclip(start_point, end_point)
		extracted_samples.append(extracted_sample)
	return extracted_samples

def combine_clip(samples):
	combined_clip = concatenate_videoclips(samples)
	print('Concatenation done')
	return combined_clip

def select_random_music(music_file_path):
	dir_folder = glob.glob(music_file_path)
	return random.choice(dir_folder)

def main(video_file_path, music_file_path, no_of_videos, output_duration):
	dir_folder = glob.glob(video_file_path)

	#随机选择并从 samples 里的文件夹里加载 n 个文件
	random_sample_paths = random.sample(dir_folder, no_of_videos)
	loaded_samples = [VideoFileClip(x) for x in random_sample_paths]

	#提取并连接这些 samples
	extracted_samples = extract_clip(loaded_samples, output_duration)
	combined_clip = combine_clip(extracted_samples)

	#为本次剪辑内容随机选取音乐
	music = select_random_music(music_file_path)
	combined_clip.audio = AudioFileClip(music).subclip(0, output_duration)

	#结果输出
	combined_clip.write_videofile('output.mp4')
	combined_clip.close()

In [ ]:
if __name__ == '__main__':
	parser = argparse.ArgumentParser(description='video_editor')
	parser.add_argument('--samples', type=int, default=10)
	parser.add_argument('--duration', type=int, default=10)
	args = parser.parse_args()
	video_file_path = input('Enter folder address of video samples: ')
	video_file_path += '\\*'
	music_file_path = input('Enter folder address of music samples: ')
	music_file_path += '\\*'
	main(video_file_path, music_file_path, args.samples, args.duration) 